In [ ]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Text preprocessing functions**

In [ ]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (EncoderDecoderModel, pipeline, AutoTokenizer, GPT2LMHeadModel, AutoConfig, AutoTokenizer, AutoModel, TFAutoModel)

In [ ]:
model_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_path, model_path)

loading configuration file https://huggingface.co/HooshvareLab/bert-fa-zwnj-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ba5343a1268e870ceacba2041556e22a4480d6af08e330b0650f96c96a5e50b4.967063c40b5c987b6e0d2a6d1944963280b9a53e5821fcb5379494a07dc9408d
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-zwnj-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

loading configuration file https://hug

###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [ ]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 25000)

In [ ]:
MAX_LEN = 170
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:10000]  # make dataset size 20000 for ease of computation

In [ ]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 39


### **building the dataset**

In [ ]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [ ]:
shuffle_pairs = create_shuffles(sents_data, 4)

In [ ]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' ارتش اسرائیل اغلب از حملات هوائی برای کشتن ستیزه گران فلسطینی در نوار غزه استفاده می\u200cکند',
 {'از غزه اغلب در برای حملات استفاده هوائی می\u200cکند فلسطینی اسرائیل ارتش گران ستیزه نوار کشتن',
  'از غزه هوائی اغلب اسرائیل حملات می\u200cکند برای در ستیزه نوار گران فلسطینی استفاده کشتن ارتش',
  'اسرائیل استفاده ستیزه غزه حملات فلسطینی اغلب در گران هوائی کشتن نوار ارتش برای از می\u200cکند',
  'اغلب ارتش ستیزه اسرائیل هوائی گران حملات استفاده در غزه کشتن برای می\u200cکند فلسطینی از نوار',
  'کشتن هوائی ستیزه در گران از حملات ارتش اغلب استفاده برای اسرائیل فلسطینی غزه می\u200cکند نوار'})

In [ ]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [ ]:
df = create_dataframe(shuffle_pairs)
df = df.sample(frac=1).reset_index(drop=True)
df

,Shuffled,Original
0,مسلمان شورشی به به گفته کمین ۲۰ حادثه او درمحل...,به گفته او نزدیک به ۲۰ مسلمان شورشی درمحل حاد...
1,,
2,انفجاریک کم اتومبیل هلاکت ودربغداد را دونفر دس...,ودربغداد انفجاریک اتومبیل حامل بمب دست کم دون...
3,خبرنگار به ضد محکوم به اتهام دادگاه کرده سال ر...,دادگاه جمهوری اسلامی خبرنگار رادیو فردا را به...
4,اتمی سازمان این بین بر وجودش از دید سازمان الم...,سازمان بین المللی انرژی اتمی میگوید که بازرسا...
...,...,...
49359,از برای کار ایران دانشجوئی شد درخواست به جوابگ...,خبرگزاری دانشجوئی ایران میگوید درخواست استیضا...
49360,نخواهد قوانین شد آقای گفت هرگز از یانکوویچ تقل...,آقای یانکوویچ گفت قوانین جدید هرگز مانع از تق...
49361,دردهلی وزیرامورخارجه پیشروی به روزدوشنبه متعهد...,ناتوارسینگ وزیرامورخارجه هند روزدوشنبه دردهلی...
49362,سودان نظامیان که شهرت کرده به را از دارند است ...,سودان حمایت از شبه نظامیان عرب را که به جانجا...


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import Dataset
def split_dataframe(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  train_data   = df.iloc[:count]
  valid_data   = df.iloc[count:]
  return train_data, valid_data

train_set, val_set = split_dataframe(df)

In [ ]:
batch_size=64 
encoder_max_length=64
decoder_max_length=64

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["Shuffled"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["Original"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
train_data = Dataset.from_pandas(train_set, split='train')

In [ ]:
batch_size = 16

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
)

  0%|          | 0/2469 [00:00<?, ?ba/s]

In [ ]:
train_data

Dataset({
    features: ['Shuffled', 'Original', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 39491
})

In [ ]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
val_data = Dataset.from_pandas(val_set, split='validation')

In [ ]:
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 

)

  0%|          | 0/618 [00:00<?, ?ba/s]

In [ ]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
bert2bert.config.encoder.vocab_size

42000

In [ ]:
bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

In [ ]:
bert2bert.config.max_length = 64
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
%%capture
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=16,  # set to 500 for full training
    eval_steps=4,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=32, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import datasets
import numpy as np
from datasets import load_metric

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics_rouge(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
metric = load_metric("accuracy")

def compute_metrics_acc(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc_output = metric.compute(predictions=predictions, references=labels)
    return {
        "accuracy": acc_output
    }



In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics_rouge,
    train_dataset=train_data,
    eval_dataset=val_data,
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 39491
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 32
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on th

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
4,0.526300,0.102377,0.000000,0.000000,0.000000
8,0.081800,0.019663,0.000000,0.000000,0.000000
12,0.044900,0.012742,0.000000,0.000000,0.000000
16,0.032000,0.009030,0.000000,0.000000,0.000000
20,0.032600,0.007990,0.000000,0.000000,0.000000
24,0.033300,0.007165,0.000000,0.000000,0.000000
28,0.015900,0.006650,0.000000,0.000000,0.000000
32,0.014400,0.006455,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 9873
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9873
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9873
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9873
  Batch size = 16
Saving model checkpoint to ./checkpoint-16
Configuration saved in ./checkpoint-16/config.json
Model weights saved in ./checkpoint-16/pytorch_model.bin
tokenizer config file saved in ./checkpoint-16/tokenizer_config.json
Special tokens file saved in ./checkpoint-16/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:533: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now crea

TrainOutput(global_step=32, training_loss=0.15949079510755837, metrics={'train_runtime': 10989.0431, 'train_samples_per_second': 0.047, 'train_steps_per_second': 0.003, 'total_flos': 39263319097344.0, 'train_loss': 0.15949079510755837, 'epoch': 0.01})

In [ ]:
test_data = Dataset.from_pandas(val_set)
test_data = test_data.select(range(16))
test_data

Dataset({
    features: ['Shuffled', 'Original'],
    num_rows: 16
})

In [ ]:
batch_size = 16  

def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Shuffled"], padding="max_length", truncation=True, max_length=64, return_tensors="pt")
    #outputs = tokenizer(batch["Original"], padding="max_length", truncation=True, max_length=64, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    #decoder_input_ids= outputs.input_ids.to("cuda")
    #decoder_attention_mask = outputs.attention_mask.to("cuda")

    outputs = bert2bert.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)

pred_str = results["pred"]
label_str = results["Original"]
print('\n--------------- predictions ---------------\n')
_ = [print(sent) for sent in pred_str]
print('\n--------------- original ---------------\n')
_ = [print(sent) for sent in label_str]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 64 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,



--------------- predictions ---------------

و و و تنظیم تنظیم تنظیم تدوین تدوین تدوین تهیه تهیه تهیه تدوین تهیه تدوین تدوین فیلمنامه فیلمنامه فیلمنامه نمایشنامه نمایشنامه فیلمنامه فیلمنامه فیلم فیلم فیلم فیلمی فیلمی فیلمی فیلم فیلم کارگردان کارگردان کارگردان فیلمساز فیلمساز فیلمساز کارگردان کارگردان فیلمسازان فیلمسازان فیلمساز فیلمساز فیلمسازان فیلمسازان فیلمسازان کارگردانان فیلمسازان فیلمسازان سینما سینما سینما فیلم فیلم سینما سینما
تمرکز تمرکز تمرکز متمرکز متمرکز متمرکز تمرکز تمرکز معطوف معطوف معطوف متمرکز معطوف متمرکز متمرکز معطوف معطوف صرف صرف صرف هزینه هزینه هزینه صرف صرف نظر نظر نظر نظرهای نظرهای نظرهای نظرات نظرات نظرات نظرهای نظرهای نظرها نظرها نظرها نظرهای نظرهای نظر نظر نظرات نظرات نظرها نظرها نظرات نظرات نظر نظر دیدگاه دیدگاه دیدگاه نظرات نظرات دیدگاه دیدگاه
و و و « « « " " " ۳ ۳ ۲ ۲ ۲ ۳ ۳ ۳ ۴ ۴ ۴ ۳ ۳ ۵ ۵ ۵ ۴ ۴ ۵ ۵ 5 5 5 ۵ ۵ ۲ ۲ ۴ ۴ ۲ ۲ ۱ ۱ ۱ ۲ ۲ ۵ ۵ ۳ ۳ ۱ ۱ ۴ ۴ ۱ ۱ ۳
و و و « « « " " " ۳ ۳ ۳ ۲ ۲ ۲ ۳ ۳ ۴ ۴ ۴ ۳ ۳ ۵ ۵ ۵ ۴ ۴ ۵ ۵ ۲ ۲ ۱ ۱ ۱ ۲ ۲ ۴ ۴ ۲ ۲ ۵ ۵ ۳ ۲ ۳ ۴ ۳ ۲ ۴ ۳ ۴ ۲ ۳ 